### Preprocessing of CDR3 with Emerson dataset

In [ ]:
path_files <- '../../Emerson_DeWitt//CMV/'
path_out <- '../../Emerson_DeWitt//preprocessed_cdr3/'
seq_depth <- 30000

if (!file.exists(path_out)) {
        dir.create(path_out, recursive = TRUE)
        }

for (f in grep('P', list.files(path_files), value = TRUE)){
    path <- paste0(path_files,f)
    sample_id <- paste0('P', as.numeric(unlist(strsplit(unlist(strsplit(f, '.tsv'))[[1]], 'P'))[[2]]))
    cdr3_dt <- fread(path)
    if (nrow(cdr3_dt) > seq_depth){ 
    cdr3_dt <- cdr3_dt[frame_type == 'In',][
         , patient_id := sample_id][
         , unique_count := 1][
         , length_seq := nchar(amino_acid)][
         , c("v_gene", "v_gene_2") := tstrsplit(v_gene, "/")][  
         , v_gene := fifelse(is.na(v_gene), v_family, v_gene)][  
         , j_gene := fifelse(is.na(j_gene), j_family, j_gene)][  
         , .(unique_count = sum(unique_count), cum_expansion_freq = sum(productive_frequency)),                             
         by = .(patient_id, amino_acid, v_gene, j_gene, length_seq)][
         (length_seq >= 12) & (length_seq <=18)]  
    fwrite(cdr3_dt, paste0(path_out,'preprocessed_',sample_id,'.tsv'), sep = '\t')
    }

}

### Removed V and J, aligned to IMGT and calculated unique and expanded frequencies for each length-position-AA

In [29]:
path_in <- '../../Emerson_DeWitt//preprocessed_cdr3/'
path_out <- '../../Emerson_DeWitt//cdr3_freq_with_expansion/'
if (!file.exists(path_out)) {
        dir.create(path_out, recursive = TRUE)
        }

cdr3_freq_output_file <- '../../Emerson_DeWitt//cdr3_all_freq_with_expansion_excluded_germ_long.tsv'
cdr3_freq_output_file_irt <- '../../Emerson_DeWitt//cdr3_all_irt_with_expansion_excluded_germ_long.tsv'

first_file <- TRUE
for (f in list.files(path_in)){
    cdr3 <- fread(paste0(path_in,f))
    id <- cdr3$patient_id[1]
    cdr3_aligned <- as.data.table(t(sapply(cdr3$amino_acid, align_imgt)))
    setnames(cdr3_aligned, all_imgt_pos)
    cdr3_aligned <- cbind(cdr3,cdr3_aligned)
    rm(cdr3)

    
    processed_cdr3 <- melt(cdr3_aligned, id.vars = c("patient_id", "amino_acid", "v_gene", "j_gene", "length_seq", "cum_expansion_freq", "unique_count"), 
                    measure.vars = all_imgt_pos, 
                    variable.name = "IMGT", 
                    value.name = "AA")
    processed_cdr3 <- na.omit(processed_cdr3)
    setDT(processed_cdr3)
    
    V_germ_line[, v_gene_family := sub("-.*", "", v_gene)]
    
    J_germ_line[, j_gene_family := sub("-.*", "", j_gene)][
        ,length_seq := as.integer(length_seq)]
    
    # Perform an anti-join to remove rows from main_dt where v_gene and j_gene or matches v_gene_family, j_gene_family and IMGT, AA match
    processed_cdr3_removing_v_j <- processed_cdr3[!V_germ_line, 
                                            on = c("v_gene", "IMGT", "AA")][!V_germ_line, 
                                            on = c("v_gene" = "v_gene_family", "IMGT", "AA")][!J_germ_line, 
                                            on = c("j_gene", "length_seq", "IMGT", "AA")][!J_germ_line, 
                                            on = c("j_gene" = "j_gene_family", "length_seq", "IMGT", "AA")][!(AA == 'NA')]
    processed_cdr3_removing_v_j <- processed_cdr3_removing_v_j[!(v_gene == 'unresolved' & IMGT %in% c('P104', 'P105', 'P106'))][!(j_gene == 'unresolved' & IMGT %in% c('P117', 'P118'))]
    
    rm(processed_cdr3)
    
    cdr3_long_freq <- processed_cdr3_removing_v_j[, .(patient_id, length_seq, IMGT, AA, cum_expansion_freq, unique_count )][, .(
        cum_expansion_freq = sum(cum_expansion_freq),  # Summing cum_expansion_freq
        unique_count = sum(unique_count)               # Summing unique_count
        ), by = .(patient_id, length_seq, IMGT, AA)]
    cdr3_long_freq <- cdr3_long_freq[, total_imgt_count := sum(unique_count),by = .(patient_id, length_seq, IMGT)][
        , norm_freq_expand := cum_expansion_freq/sum(cum_expansion_freq),by = .(patient_id, length_seq, IMGT)][
        , norm_freq_unique := unique_count/total_imgt_count, by = .(patient_id, length_seq, IMGT)]
    
    fwrite(cdr3_long_freq, cdr3_freq_output_file, 
           sep = '\t', quote = FALSE, row.names = FALSE, append = TRUE, 
           col.names = first_file)
    fwrite(cdr3_aligned, paste0(path_out,'../CDR3/cdr3_wide_aligned_imgt.tsv'), sep = '\t',
           quote = FALSE, row.names = FALSE, append = TRUE, 
           col.names = first_file)
    first_file <- FALSE

    
    #-----remove variables to clean the space----
    rm(processed_cdr3_removing_v_j)
    rm(cdr3_long_freq)
    rm(cdr3_aligned)
    }


cdr3_freq <- fread(cdr3_freq_output_file)

cdr3_freq <- cdr3_freq[(total_imgt_count > 20)]
cdr3_freq <- cdr3_freq[, `:=`(
    n_carriers = uniqueN(patient_id),
    n_ind_high_expand_freq = sum(norm_freq_expand > 0.6),
    irt_freq_unique = qnorm((rank(norm_freq_unique, na.last="keep") - 0.5) / sum(!is.na(norm_freq_unique))),
    irt_freq_expand = qnorm((rank(norm_freq_expand, na.last="keep") - 0.5) / sum(!is.na(norm_freq_expand)))
    ), by = .(length_seq, IMGT, AA)]

fwrite(cdr3_freq, cdr3_freq_output_file_irt, sep = '\t', quote = FALSE, row.names = FALSE)


### Prepate CDR3_HLA matrixes with IRT for AA which are present in almost everyone (to keep normality)

#### HLA with 3PCs (removing correlated alleles > 0.5)

In [30]:
source('R_functions/hla_functions.R')

In [ ]:
hla_genes = c('DRB1','DQA1','DQB1','A','B','C','DPB1', 'DPA1')

In [ ]:
cdr3_inds <- unlist(lapply(unlist(list.files('../../Emerson_DeWitt//preprocessed_cdr3/')), function(x) gsub('preprocessed_', '', gsub('.tsv', '', x))))
write.table(cdr3_inds, '../../Emerson_DeWitt//inds_in_cdr3.txt', row.names = FALSE, quote = FALSE)  

ind_in_cdr3 <- unique(fread('../../Emerson_DeWitt//inds_in_cdr3.txt')$x)
hla_features <- fread('../../Emerson_DeWitt/hla/hla_features_DeWitt.tsv')
hla_features[patient_id < 81, patient_id := patient_id +1 ]
hla_features[patient_id > 81, patient_id := patient_id +2 ]
hla_features[, patient_id := paste0('P', patient_id)]
hla_features <- hla_features[patient_id %in% ind_in_cdr3]

fwrite(hla_features, '../../Emerson_DeWitt/hla/hla_features_matched_DeWitt.tsv', sep = '\t')

hla_genotypes_matrix <- as.data.table(dcast(hla_features, patient_id ~ allele, value.var = 'homo_hetero', fill = 0))
                           
low_ab_alleles <- names(colSums(hla_genotypes_matrix[,-1])[which(colSums(hla_genotypes_matrix[,-1]) < round(n_alleles_per_gene * 0.01))])
hla_genotypes_matrix_wo_low_ab <- hla_genotypes_matrix[, c(low_ab_alleles):=NULL]
pca_hla <- pca_hla_fun(hla_genotypes_matrix_wo_low_ab, 0.5, 3)
fwrite(pca_hla, '../../Emerson_DeWitt/hla/hla_pca_DeWitt.tsv', sep = '\t')
                           
for (gene_name in hla_genes){   
 
    hla_patients_proper_sites <- patient_site_variation_fun(hla_features, gene_name)
    
    site_split <- hla_patients_proper_sites %>% 
        dplyr::select(patient_id, homo_hetero, site, AA) %>% 
        group_split(site)
    
    for (site in seq(1, length(site_split))){
        site_location <- site_split[[site]]$site[1]
        site_matrix <- site_split[[site]] %>% dplyr::select(-site) %>% 
            pivot_wider(names_from=AA, values_from = homo_hetero, values_fn = sum, values_fill = 0, names_prefix = 'allele_') %>%
            inner_join(pca_hla, by = 'patient_id') 
        site_matrix$G_S <- paste0(gene_name,'_',site_location)
        write_tsv(site_matrix, paste0("../../Emerson_DeWitt/hla_matrices/",gene_name, "_",site_location, "_matrix.tsv"))
    }
    

}                           


In [ ]:
ggplot(pca_hla, aes(x = PC1, y = PC2)) +
    geom_point(alpha = 0.5) +
    #scale_color_manual(values = c('healthy' = "#673AB7", 'IBD' = "#FF6F61" )) +
    theme_cowplot() + 

      theme(
    plot.title = element_text(size = 12),         # Adjust title size
    axis.title = element_text(size = 10),         # Adjust axis title size
    axis.text = element_text(size = 8),           # Adjust axis label size
    legend.title = element_text(size = 10),       # Adjust legend title size
    legend.text = element_text(size = 8)          # Adjust legend text size
  )

In [32]:
cdr3_freq

patient_id,length_seq,IMGT,AA,cum_expansion_freq,unique_count,total_imgt_count,norm_freq_expand,norm_freq_unique,n_carriers,n_ind_high_expand_freq,irt_freq_unique,irt_freq_expand
<chr>,<int>,<chr>,<chr>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>
P1,13,P104,R,4.631218e-06,6,26,0.060483871,0.23076923,571,110,-0.98628558,-1.26770399
P1,14,P104,A,1.130017e-04,43,77,0.393125671,0.55844156,142,0,2.69483954,2.69483954
P1,13,P104,Y,1.790738e-05,7,26,0.233870968,0.26923077,559,0,1.71761085,1.27545939
P1,12,P104,R,4.631218e-06,5,35,0.055350554,0.14285714,456,37,-1.57080448,-1.40477660
P1,13,P104,F,6.174957e-06,5,26,0.080645161,0.19230769,415,2,1.12014823,0.63150429
P1,12,P104,F,5.279588e-05,26,35,0.630996310,0.74285714,389,1,3.01489361,3.01489361
P1,14,P104,R,1.234991e-05,7,77,0.042964554,0.09090909,563,195,-2.61660154,-2.16751796
P1,14,P104,G,1.296741e-05,9,77,0.045112782,0.11688312,391,1,1.64609479,0.68458452
P1,14,P104,V,6.174957e-06,1,77,0.021482277,0.01298701,77,0,-0.68474239,0.03255937


In [33]:
cdr3_freq <- cdr3_freq[(total_imgt_count > 20 & n_carriers > 200)]
cdr3_freq_split_l_imgt <- split(cdr3_freq, by = c('length_seq', 'IMGT'))

dir_pair_path <- paste0('../../Emerson_DeWitt/cdr3_hla_pairs/')
hla_matrices <- paste0('../../Emerson_DeWitt/hla_matrices/',list.files('../../Emerson_DeWitt/hla_matrices/'))
path_group <- paste0(dir_pair_path, '/CDR3_PAIRS_UNIQUE_IRT/') 
if (!file.exists(path_group)) {
        dir.create(path_group, recursive = TRUE)
        }

prep_modes <- c('irt_freq_unique')

for (L_P_group in seq(1,length(cdr3_freq_split_l_imgt))){

    cdr3_L_P <- cdr3_freq_split_l_imgt[[L_P_group]]
   
    L <- unique(cdr3_L_P$length_seq)
    P <- unique(cdr3_L_P$IMGT)
        
    cdr3_group_long <- cdr3_prep_for_irt_matrix_fun(cdr3_L_P)    
    

    fwrite(cdr3_group_long, paste0(path_group,'../CDR3_LONG_UNIQUE_IRT/',L,'_',P,'.tsv'), sep = '\t')
    

    for (prep_mode in prep_modes){
        cdr3_matrix <- dcast(cdr3_group_long, patient_id ~ AA, value.var = prep_mode)
            
        for (f in hla_matrices){
            
            hla_matrix <- fread(f)
            
            cdr3_hla_matrix <- merge(hla_matrix, cdr3_matrix, by = 'patient_id', all.x = TRUE)
            cdr3_hla_matrix$pair <- paste0(hla_matrix$G_S[1],'_', L, '_', P)
            name_pair <- cdr3_hla_matrix$pair[1]
    
            cdr3_hla_matrix[, G_S := NULL]
            fwrite(cdr3_hla_matrix, paste0(path_group,name_pair,'_matrix.tsv'), sep = '\t')
            
        }
        }

    
}

In [39]:
cdr3_hla_matrix

patient_id,allele_E,allele_K,PC1,PC2,PC3,C,R,Y,pair
<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
P1,0,2,-0.0645195668,-0.029414669,-0.0726282310,NA,NA,NA,DRB1_98_17_P104
P10,0,2,-0.0239687926,0.091461494,0.0282618329,NA,NA,NA,DRB1_98_17_P104
P100,1,1,-0.0506747784,-0.034453326,-0.0099632944,NA,NA,NA,DRB1_98_17_P104
P101,0,2,-0.0063151200,-0.016344517,-0.0046301248,NA,NA,NA,DRB1_98_17_P104
P102,1,1,-0.0092075833,-0.036762611,0.0751439629,NA,NA,NA,DRB1_98_17_P104
P103,1,1,-0.0129384556,-0.037626511,0.0856131290,NA,NA,NA,DRB1_98_17_P104
P105,1,1,-0.0475691011,-0.039192888,0.0000764782,NA,NA,NA,DRB1_98_17_P104
P106,0,2,-0.0335456630,0.097439412,0.0232263109,-0.7924983,1.24660177,-0.6463198,DRB1_98_17_P104
P107,1,1,-0.0437991471,0.049694480,-0.0016965427,NA,NA,NA,DRB1_98_17_P104


In [101]:
n_tests <- length(list.files('../../Emerson_DeWitt/cdr3_hla_pairs/CDR3_PAIRS_UNIQUE_IRT/'))

In [4]:
source('R_functions//cdr3-QTL_functions.R')

In [ ]:
hla_features <- fread('../../Emerson_DeWitt/hla/hla_features_matched_DeWitt.tsv')
pairs <- '../../Emerson_DeWitt/cdr3_hla_pairs/CDR3_PAIRS_UNIQUE_IRT/'
dir_pair_path <- paste0(pairs, list.files(pairs))
                           
dir_results <- '../../Emerson_DeWitt/manova_results/'
pcs <- 3

first <- TRUE
for (f in dir_pair_path){
    
    cdr3_hla_matrix <- na.omit(fread(f)) 
    name_pair <- cdr3_hla_matrix$pair[1]
    
    manova_results <- mlm_fun(cdr3_hla_matrix, dir_results, n_pcs = pcs)
    if (first){
        manova_df_all <- manova_results[0,]
        first <- FALSE
    }
    if (is.null(manova_results)){
        next
    } else {
        manova_df_all <- rbind(manova_df_all, manova_results) 
        
    }
  
    }
manova_df_all <- manova_df_all %>% separate(pair, into = c('HLA', 'Site_hla', 'Length_cdr3', 'Position_cdr3'), sep = '_', remove = FALSE)

fwrite(manova_df_all, paste0(dir_results, 'manova_results_',pcs,'PCs.tsv'), sep = '\t')

Warning message in mvlm(formula_full, cdr3_hla_matrix):
“Adjusted sample size = 54
Asymptotic properties of the null distribution may not hold.
This can result in overly conservative p-values.
Increased sample size is recommended.”
Warning message in mvlm(formula_null, cdr3_hla_matrix):
“Adjusted sample size = 54
Asymptotic properties of the null distribution may not hold.
This can result in overly conservative p-values.
Increased sample size is recommended.”
Warning message in mvlm(formula_full, cdr3_hla_matrix):
“Adjusted sample size = 57
Asymptotic properties of the null distribution may not hold.
This can result in overly conservative p-values.
Increased sample size is recommended.”
Warning message in mvlm(formula_null, cdr3_hla_matrix):
“Adjusted sample size = 58
Asymptotic properties of the null distribution may not hold.
This can result in overly conservative p-values.
Increased sample size is recommended.”
Warning message in mvlm(formula_full, cdr3_hla_matrix):
“Adjusted sample

### Conditional on Emerson dataset

In [ ]:
source('/work_beegfs/sukmb667/projects/cdr3-qtl/healthy_and_ibd/scripts/R_scripts/Conditional_haplotype_HLA_tutorial.R')


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘tidyr’


The following object is masked from ‘package:magrittr’:

    extract


Loading required package: ggplot2

Loading required package: lattice


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


The following objects are masked from ‘package:data.table’:

    dcast, melt



---------------------
Welcome to dendextend version 1.17.1
Type citation('dendextend') for how to cite the package.

Type browseVignettes(package = 'dendextend') for the package vignette.
The github page is: https://github.com/talgalili/dendextend/

Suggestions and bug-reports can be submitted at: https://github.com/talgalili/dendextend/issues
You may ask qu